Part 1

In [ ]:
import random

Generate Distribution Functions

In [10]:
import random

def sorted_indexes(payoffs):
    vals_indexes=[]
    ind_by_val = []

    for i in range(len(payoffs)):
        vals_indexes.append([payoffs[i],i])
    
    vals_indexes.sort(reverse=True)
    for x in vals_indexes:
        ind_by_val.append(x[1])
    return ind_by_val

def find_min_index(payoffs):
    min_value = min(payoffs)
    min_index = payoffs.index(min_value)
    return min_index

def generate_adversarial_payoffs(num_actions, num_rounds):
    rounds_list = []
    totals_by_round = []
    #first_payoffs = [round(random.random(),2) for i in range(num_actions)]
    initial_payoff = round(random.random(), 2)
    first_payoffs = [0 for i in range(num_actions)]
    first_payoffs[random.randrange(num_actions)] = initial_payoff
    total_payoffs = [first_payoffs[i] for i in range(num_actions)]
    #total_indexes = sorted_indexes(total_payoffs)
    min_index = find_min_index(total_payoffs)
    rounds_list.append(first_payoffs)
    totals_by_round.append([total_payoffs[i] for i in range(num_actions)])
    
    for i in range(num_rounds - 1):
        #print("initial total payoffs", total_payoffs)
        #new_payoffs = sorted([round(random.random(),2) for i in range(num_actions)])
        new_payoff = round(random.random(), 2)
        adversarial_payoffs = [0 for i in range(num_actions)]
        #for i in range(num_actions):
        #   adversarial_payoffs[total_indexes[i]] = new_payoffs[i]
        adversarial_payoffs[min_index] = new_payoff
        for i in range(num_actions):
            #print("t", total_payoffs[i], "a", adversarial_payoffs[i])
            total_payoffs[i] += adversarial_payoffs[i]
            total_payoffs[i] = round(total_payoffs[i], 2)
        
        #total_indexes = sorted_indexes(total_payoffs)
        min_index = find_min_index(total_payoffs)
        new_totals = [total_payoffs[i] for i in range(num_actions)]
        totals_by_round.append(new_totals)
        #print(new_totals)
        rounds_list.append(adversarial_payoffs)

    print("utility at each round: \n", rounds_list)
    print("totals by round: \n", totals_by_round)
    print("final payoffs: \n", total_payoffs)
    return rounds_list, totals_by_round


generate_adversarial_payoffs(3, 3)

utility at each round: 
 [[0, 0.4, 0], [0.15, 0, 0], [0, 0, 0.29]]
totals by round: 
 [[0, 0.4, 0], [0.15, 0.4, 0], [0.15, 0.4, 0.29]]
final payoffs: 
 [0.15, 0.4, 0.29]


([[0, 0.4, 0], [0.15, 0, 0], [0, 0, 0.29]],
 [[0, 0.4, 0], [0.15, 0.4, 0], [0.15, 0.4, 0.29]])

In [8]:
import random
#when generating the bernoulli payoffs, generate the payoffs of each action at each round and the 
#total payoffs up to that point for each action. i.e. list of lists of payoffs/round & list of lists of aggregated payoffs.
#uncomment the last line of the generate_adversarial_payoffs section for an example
def find_payoff(success_chance):
    comparison_val = random.random()
    return int(success_chance > comparison_val)

def generate_bernoulli_payoffs(num_actions, num_rounds):
    rounds_list = []
    totals_by_round = []
    total_payoffs = [0 for i in range(num_actions)]
    totals_by_round = []
    
    #generate probability of action success in range (0, 1/2)
    action_success_chances = [round(random.random() / 2,2) for i in range(num_actions)]
    print(action_success_chances)
    
    for i in range(num_rounds - 1):
        #print("initial total payoffs", total_payoffs)
        new_payoffs = [find_payoff(action_success_chances[j]) for j in range(num_actions)]
        print(new_payoffs)
        
        for i in range(num_actions):
            #print("t", total_payoffs[i], "a", adversarial_payoffs[i])
            total_payoffs[i] += new_payoffs[i]
            total_payoffs[i] = round(total_payoffs[i], 2)
        
        new_totals = [total_payoffs[i] for i in range(num_actions)]
        totals_by_round.append(new_totals)
        #print(new_totals)
        rounds_list.append(new_payoffs)

    #print("utility at each round: \n", rounds_list)
    #print("totals by round: \n", totals_by_round)
    #print("final payoffs: \n", total_payoffs)
    return rounds_list, totals_by_round

#generate_bernoulli_payoffs(3, 3)

def generate_random_payoffs(num_actions, num_rounds):
    pass

Simulate Algorithm Behavior Functions

In [96]:
def simulate_random_guessing(rounds_list, totals_by_round):
    return simulate_exponential_weights(rounds_list, totals_by_round, 0)


def simulate_follow_leader(round, inputs):
    #may need to implement this manually.
    return simulate_exponential_weights(rounds_list, totals_by_round, 100000)

def simulate_exponential_weights(rounds_list, totals_by_round, epsilon):
    num_rounds = len(rounds_list)
    num_actions = len(rounds_list[0])
    choices_made = []
    action_weights = []
    action_probabilities = [(1/num_actions) for i in range(num_actions)]
    current_weights = [1 for i in range(num_actions)]
    action_weights.append(current_weights)
    alg_payoffs = []
    opt_payoffs = []
    
    for round in range(1, num_rounds):
        last_round = round - 1
        current_weights = [None for i in range(num_actions)]
        for action in range(num_actions):
            V_last = totals_by_round[last_round][action]
            exp = V_last / num_actions
            current_weights[action] = pow(1 + epsilon, exp)
        #randomly select from actions using weights as probabilities
        selected_payoff = random.choices(rounds_list[round], weights=current_weights, k=1)[0]
        print(current_weights)
        alg_payoffs.append(selected_payoff)  
        opt_payoffs.append(max(rounds_list[round]))
        action_weights.append(current_weights)
        
    print("EW alg payoff", sum(alg_payoffs))
    print("OPT payoff", max(totals_by_round[-1]))
    return

rounds_list_1, payoff_totals_1 = generate_adversarial_payoffs(3, 10)
epsilon_1 = 1
simulate_exponential_weights(rounds_list_1, payoff_totals_1, epsilon_1)




utility at each round: 
 [[0.89, 0.03, 0.86], [0.08, 0.34, 0.18], [0.51, 0.79, 0.17], [0.2, 0.71, 0.65], [0.87, 0.25, 0.45], [0.79, 0.93, 0.91], [0.25, 0.92, 0.58], [0.65, 0.04, 0.1], [0.19, 0.21, 0.68], [0.2, 0.85, 0.16]]
totals by round: 
 [[0.89, 0.03, 0.86], [0.97, 0.37, 1.04], [1.48, 1.16, 1.21], [1.68, 1.87, 1.86], [2.55, 2.12, 2.31], [3.34, 3.05, 3.22], [3.59, 3.97, 3.8], [4.24, 4.01, 3.9], [4.43, 4.22, 4.58], [4.63, 5.07, 4.74]]
final payoffs: 
 [4.63, 5.07, 4.74]
[1.2283031493691747, 1.0069555500567189, 1.219818639760204]
[1.2512181394937498, 1.0892486561426122, 1.271619166385803]
[1.4076935840339895, 1.307369247202105, 1.3225601461225394]
[1.4742692172911012, 1.5404302223135504, 1.5368751812880124]
[1.8025009252216604, 1.632028969976891, 1.7052697835359134]
[2.1634493321602095, 2.023238880603845, 2.104289696401433]
[2.2920947239400062, 2.5024362789874997, 2.4060500721642333]
[2.663518558843832, 2.525670902383281, 2.4622888266898326]
[2.7830496883568325, 2.6512388835730567, 2.

Monte Carlo Trials

- Declare size of inputs
- Generate payoffs
- For each learning rate $\{\epsilon_1, . . ., \epsilon_n\}$
    - For each input
        - Simulate the algorithm
        - calculate OPT (best in hindsight payoff)
        - calculate the algorithm's regret
    - Calculate the average regret for this learning rate $\epsilon$

In [8]:
rounds = 100
n = 5